In [3]:
import tensorflow as tf

In [4]:
tf.enable_eager_execution()

## Conv test

In [53]:
s=[[1.0,2.0,3.0,4.0,5.0,6.0],[10.0,20.0,30.0,40.0,50.0,60.0]]

In [54]:
s = tf.square(s)

In [55]:
s.shape

TensorShape([Dimension(2), Dimension(6)])

In [56]:
a = [[1.0,2.0,3.0,4.0,5.0,6.0],[10.0,20.0,30.0,40.0,50.0,60.0],[100.0,200.0,300.0,400.0,500.0,600.0],
     [-1.0,-20.,-3.0,-4.0,-5.0,-6.0],[-10.0,-20.0,-30.0,-40.0,-50.0,-60.0],[-100.0,-200.0,-300.0,-400.0,-500.0,-600.0]]

In [57]:
a = tf.expand_dims(a, axis=0)
a = tf.expand_dims(a, axis=3)

In [59]:
tf.squeeze(a)

<tf.Tensor: id=215, shape=(6, 6), dtype=float32, numpy=
array([[   1.,    2.,    3.,    4.,    5.,    6.],
       [  10.,   20.,   30.,   40.,   50.,   60.],
       [ 100.,  200.,  300.,  400.,  500.,  600.],
       [  -1.,  -20.,   -3.,   -4.,   -5.,   -6.],
       [ -10.,  -20.,  -30.,  -40.,  -50.,  -60.],
       [-100., -200., -300., -400., -500., -600.]], dtype=float32)>

In [60]:
a[0][1][:]

<tf.Tensor: id=228, shape=(6, 1), dtype=float32, numpy=
array([[10.],
       [20.],
       [30.],
       [40.],
       [50.],
       [60.]], dtype=float32)>

In [61]:
kernel = [[1.0,1.0,1.0],[1.0,1.0,1.0],[1.0,1.0,1.0]]
kernel = tf.expand_dims(kernel, axis=2)
kernel = tf.expand_dims(kernel, axis=3)

In [62]:
tf.nn.conv2d(a, kernel, strides=[1,2,2,1], padding="VALID")

<tf.Tensor: id=235, shape=(1, 2, 2, 1), dtype=float32, numpy=
array([[[[ 666.],
         [1332.]],

        [[ 516.],
         [1068.]]]], dtype=float32)>

In [68]:
kernel_2 = [[1.0,1.0,1.0,1.0],[1.0,1.0,1.0,1.0],[1.0,1.0,1.0,1.0],[1.0,1.0,1.0,1.0]]
kernel_2 = tf.expand_dims(kernel_2, axis=2)
kernel_2 = tf.expand_dims(kernel_2, axis=3)

In [70]:
tf.squeeze(tf.nn.conv2d(a, kernel_2, strides=[1,1,1,1], padding="VALID"))

<tf.Tensor: id=254, shape=(3, 3), dtype=float32, numpy=
array([[1082., 1522., 1980.],
       [ 972., 1368., 1782.],
       [-128., -172., -198.]], dtype=float32)>

## Conv test 2

In [3]:
input_4x4 = tf.ones([4,4])
input_4x4 = tf.expand_dims(input_4x4, axis=0)
input_4x4 = tf.expand_dims(input_4x4, axis=3)

In [4]:
input_4x4

<tf.Tensor: id=7, shape=(1, 4, 4, 1), dtype=float32, numpy=
array([[[[1.],
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         [1.]]]], dtype=float32)>

In [5]:
kernel_1 = tf.ones([3,3,2])*2
kernel_1 = tf.expand_dims(kernel_1, axis=2)
#kernel = tf.expand_dims(kernel, axis=3)

In [6]:
kernel_1

<tf.Tensor: id=14, shape=(3, 3, 1, 2), dtype=float32, numpy=
array([[[[2., 2.]],

        [[2., 2.]],

        [[2., 2.]]],


       [[[2., 2.]],

        [[2., 2.]],

        [[2., 2.]]],


       [[[2., 2.]],

        [[2., 2.]],

        [[2., 2.]]]], dtype=float32)>

In [7]:
result = tf.nn.conv2d(input_4x4, kernel_1, strides=[1,1,1,1], padding="SAME")

In [8]:
tf.transpose(tf.squeeze(result), perm=[2,0,1])

<tf.Tensor: id=19, shape=(2, 4, 4), dtype=float32, numpy=
array([[[ 8., 12., 12.,  8.],
        [12., 18., 18., 12.],
        [12., 18., 18., 12.],
        [ 8., 12., 12.,  8.]],

       [[ 8., 12., 12.,  8.],
        [12., 18., 18., 12.],
        [12., 18., 18., 12.],
        [ 8., 12., 12.,  8.]]], dtype=float32)>

In [11]:
kernel_2 = tf.ones([3,3,2,2])*1
#kernel = tf.expand_dims(kernel, axis=2)

In [13]:
result_2 = tf.squeeze(tf.nn.conv2d(result, kernel_2, strides=[1,1,1,1], padding="SAME"))

In [14]:
tf.transpose(tf.squeeze(result_2), perm=[2,0,1])

<tf.Tensor: id=37, shape=(2, 4, 4), dtype=float32, numpy=
array([[[100., 160., 160., 100.],
        [160., 256., 256., 160.],
        [160., 256., 256., 160.],
        [100., 160., 160., 100.]],

       [[100., 160., 160., 100.],
        [160., 256., 256., 160.],
        [160., 256., 256., 160.],
        [100., 160., 160., 100.]]], dtype=float32)>

## Kernel initialization

In [109]:
class ICNR:
    """ICNR initializer for checkerboard artifact free sub pixel convolution
    Ref:
     [1] Andrew Aitken et al. Checkerboard artifact free sub-pixel convolution
     https://arxiv.org/pdf/1707.02937.pdf)
    Args:
    initializer: initializer used for sub kernels (orthogonal, glorot uniform, etc.)
    scale: scale factor of sub pixel convolution
    """

    def __init__(self, initializer, scale=1):
        self.scale = scale
        self.initializer = initializer

    def __call__(self, shape, dtype, partition_info=None):
        shape = list(shape)
        if self.scale == 1:
            return self.initializer(shape)

        new_shape = shape[:3] + [shape[3] // (self.scale ** 2)]
        print(new_shape)
        x = self.initializer(new_shape, dtype, partition_info)
        x = tf.transpose(x, perm=[2, 0, 1, 3])
        x = tf.image.resize_nearest_neighbor(x, size=(shape[0] * self.scale, shape[1] * self.scale))
        x = tf.space_to_depth(x, block_size=self.scale)
        x = tf.transpose(x, perm=[1, 2, 0, 3])
        return x

In [3]:
i = tf.transpose(tf.contrib.layers.xavier_initializer()([3,3,2]), perm=[2,0,1])

In [12]:
i[0]

<tf.Tensor: id=53, shape=(3, 3), dtype=float32, numpy=
array([[ 0.33405572,  0.42739612, -0.13825992],
       [ 0.2328617 , -0.18403801, -0.25328264],
       [-0.5492743 ,  0.29721463,  0.5242116 ]], dtype=float32)>

In [13]:
i[1]

<tf.Tensor: id=58, shape=(3, 3), dtype=float32, numpy=
array([[ 0.10076243, -0.19758129,  0.563247  ],
       [ 0.41506392,  0.43393964, -0.3257768 ],
       [ 0.34591526, -0.21252042,  0.13297117]], dtype=float32)>

In [110]:
ii = ICNR(tf.contrib.layers.xavier_initializer(), 2)([3,3,2,4], dtype=tf.float32)

[3, 3, 2, 1]


In [112]:
ii

<tf.Tensor: id=501, shape=(3, 3, 2, 4), dtype=float32, numpy=
array([[[[-0.17929077, -0.17929077, -0.17929077, -0.17929077],
         [-0.06559163, -0.06559163, -0.06559163, -0.06559163]],

        [[-0.34732544, -0.34732544, -0.34732544, -0.34732544],
         [ 0.23248968,  0.23248968,  0.23248968,  0.23248968]],

        [[-0.09127253, -0.09127253, -0.09127253, -0.09127253],
         [ 0.33163288,  0.33163288,  0.33163288,  0.33163288]]],


       [[[ 0.10431412,  0.10431412,  0.10431412,  0.10431412],
         [-0.10867593, -0.10867593, -0.10867593, -0.10867593]],

        [[ 0.3513302 ,  0.3513302 ,  0.3513302 ,  0.3513302 ],
         [-0.03575906, -0.03575906, -0.03575906, -0.03575906]],

        [[ 0.29061893,  0.29061893,  0.29061893,  0.29061893],
         [ 0.07268545,  0.07268545,  0.07268545,  0.07268545]]],


       [[[ 0.00367993,  0.00367993,  0.00367993,  0.00367993],
         [ 0.17776361,  0.17776361,  0.17776361,  0.17776361]],

        [[ 0.29847756,  0.29847756,  0

## Sub-Pixel Convolutional Neural Network

In [115]:
input = tf.constant([[[[1.0,1.1,1.2,1.3], [2.0,2.1,2.2,2.3]],[[3.0,3.1,3.2,3.3], [4.0,4.1,4.2,4.3]]]])

In [116]:
input_x_4 = input

In [134]:
input_x_4

<tf.Tensor: id=504, shape=(1, 2, 2, 4), dtype=float32, numpy=
array([[[[1. , 1.1, 1.2, 1.3],
         [2. , 2.1, 2.2, 2.3]],

        [[3. , 3.1, 3.2, 3.3],
         [4. , 4.1, 4.2, 4.3]]]], dtype=float32)>

In [159]:
tf.squeeze(tf.depth_to_space(input_x_4, 2))

<tf.Tensor: id=700, shape=(4, 4), dtype=float32, numpy=
array([[1. , 1.1, 2. , 2.1],
       [1.2, 1.3, 2.2, 2.3],
       [3. , 3.1, 4. , 4.1],
       [3.2, 3.3, 4.2, 4.3]], dtype=float32)>

In [391]:
def _phase_shift(I, r, axis=1):
    # Helper function with main phase shift operation
    bsize, a, b, c = I.get_shape().as_list()
    assert c % r == 0
    remainder = c/r    
    X = tf.reshape(I, (bsize, a, b, r, remainder))
    X = tf.split(X, r, axis=3)  # a, [bsize, b, r, r]
    X = tf.concat([tf.squeeze(x) for x in X], axis=axis)  # bsize, b, a*r, r
    if axis == 1:
        a = a*r
    elif axis == 2:
        b=b*r
    else:
        raise NotImplementedError
    return tf.reshape(X, (bsize, a, b, remainder))

def upsample_ps(X, r_h=1, r_w=1):   
    X = _phase_shift(X, r_w, axis=2)
    X = _phase_shift(X, r_h, axis=1)
    return X

In [392]:
tf.squeeze(PS(input_x_4,2,2))

<tf.Tensor: id=2801, shape=(4, 4), dtype=float32, numpy=
array([[1. , 1.2, 2. , 2.2],
       [1.1, 1.3, 2.1, 2.3],
       [3. , 3.2, 4. , 4.2],
       [3.1, 3.3, 4.1, 4.3]], dtype=float32)>

In [393]:
tf.squeeze(PS(input_x_4,1,2))

<tf.Tensor: id=2824, shape=(2, 4, 2), dtype=float32, numpy=
array([[[1. , 1.1],
        [1.2, 1.3],
        [2. , 2.1],
        [2.2, 2.3]],

       [[3. , 3.1],
        [3.2, 3.3],
        [4. , 4.1],
        [4.2, 4.3]]], dtype=float32)>

In [394]:
tf.squeeze(PS(input_x_4,2,1))

<tf.Tensor: id=2845, shape=(4, 2, 2), dtype=float32, numpy=
array([[[1. , 1.1],
        [2. , 2.1]],

       [[1.2, 1.3],
        [2.2, 2.3]],

       [[3. , 3.1],
        [4. , 4.1]],

       [[3.2, 3.3],
        [4.2, 4.3]]], dtype=float32)>

## Noise test

In [18]:
tf.keras.utils.normalize(tf.random_normal([3,3], name='z0', dtype=tf.float32))

<tf.Tensor: id=99, shape=(3, 3), dtype=float32, numpy=
array([[-0.1733081 , -0.49295354,  0.8526202 ],
       [ 0.5798403 , -0.11558441,  0.8064895 ],
       [ 0.04103291, -0.0288446 ,  0.9987414 ]], dtype=float32)>

In [13]:
tf.keras.utils.normalize(tf.random_normal([8, 128], name='z0', dtype=tf.float32))

<tf.Tensor: id=86, shape=(8, 128), dtype=float32, numpy=
array([[ 0.03334369, -0.03065756,  0.19843636, ...,  0.17525569,
        -0.04270612,  0.10149204],
       [ 0.12187111, -0.08422742,  0.07488698, ..., -0.03192599,
        -0.03220674, -0.04852334],
       [-0.03209723, -0.01738006, -0.18505864, ...,  0.02979561,
        -0.0852069 ,  0.0940572 ],
       ...,
       [ 0.02103861, -0.11842715, -0.03833785, ..., -0.03886202,
        -0.03241563,  0.12322578],
       [ 0.09910917, -0.11831503, -0.04994023, ..., -0.08249813,
        -0.05943928, -0.10112263],
       [ 0.04425624, -0.02504752,  0.02116181, ..., -0.10031912,
         0.1784842 ,  0.07876976]], dtype=float32)>

In [12]:
tf.nn.tanh(tf.random_normal([8, 8],dtype=tf.float32))

<tf.Tensor: id=76, shape=(8, 8), dtype=float32, numpy=
array([[ 0.9376429 , -0.6114712 , -0.22678488,  0.5250845 , -0.9419259 ,
        -0.27633157,  0.68459785, -0.03135049],
       [ 0.42617553, -0.41427484, -0.96308625, -0.96106595, -0.9099926 ,
        -0.96005845,  0.99580806,  0.07373111],
       [-0.716177  ,  0.11160462,  0.04083412, -0.95256644,  0.15062657,
        -0.6016784 , -0.90315837,  0.6733507 ],
       [-0.10969668, -0.8416044 , -0.29712316, -0.98418516, -0.94672143,
        -0.42189732,  0.16730301, -0.87664896],
       [-0.34989217, -0.23602396, -0.22283761,  0.7627271 , -0.2914926 ,
         0.5542742 ,  0.7353027 ,  0.63773686],
       [ 0.01621382, -0.4598755 ,  0.6351701 , -0.18323036,  0.00471828,
        -0.75313187,  0.35592487,  0.5342378 ],
       [ 0.7636148 ,  0.08106321, -0.3997486 ,  0.39319247,  0.0805719 ,
         0.825248  , -0.9638635 ,  0.2682678 ],
       [-0.42851612, -0.46016702, -0.7868424 , -0.24242462, -0.43853644,
         0.71402186, -0.8

In [64]:
tf.random_normal(
                [10],
                mean=0.0,
                stddev=0.75,
                dtype=tf.float32)

<tf.Tensor: id=270, shape=(10,), dtype=float32, numpy=
array([ 0.20507137,  1.2693267 ,  0.5304849 , -1.5551072 , -0.6157462 ,
        0.9527777 ,  0.18021934, -0.73273695, -0.6860755 , -1.2645137 ],
      dtype=float32)>

In [92]:
tf.truncated_normal([10],
                mean=0.004996402837218333,
                stddev=0.00906228284522334/2,
                dtype=tf.float32)
#MEAN:  0.004996402837218333
#stddev:  0.00906228284522334
#[-0.00406588  0.01405869]

<tf.Tensor: id=413, shape=(10,), dtype=float32, numpy=
array([ 0.00508721,  0.00151468, -0.01041873,  0.00784408, -0.01173892,
        0.00415978,  0.00504796,  0.00296952,  0.00982569, -0.00112459],
      dtype=float32)>

## Web crawler

In [ ]:
all_classes = ["2.1.1.185", "2.1.1.200", "2.1.1.206", "2.1.1.207", "2.1.1.220", "2.1.1.228", "2.1.1.257", "2.1.1.33", 
"2.1.1.45", "2.1.1.98", "2.1.2.11", "2.3.1.129", "2.3.1.210", "2.3.1.251", "2.3.1.82", "2.3.1.n2", "2.4.2.1", "2.4.2.10",
"2.4.2.22", "2.4.2.3", "2.4.2.44", "2.4.2.7", "2.4.2.8", "2.4.2.9", "2.4.99.17", "2.5.1.129", "2.5.1.18", "2.5.1.31", 
"2.5.1.78", "2.5.1.89", "2.5.1.n9", "2.6.99.2", "2.7.1.21", "2.7.1.33", "2.7.1.71", "2.7.2.1", "2.7.2.3", "2.7.4.10", 
"2.7.4.14", "2.7.4.22", "2.7.4.26", "2.7.4.3", "2.7.4.6", "2.7.7.2", "2.7.7.3", "2.7.7.56", "2.7.7.60", "2.7.7.68", 
"2.7.7.77", "2.8.1.10"]

In [9]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [32]:
from bs4 import BeautifulSoup
for enzyme_class in all_classes:
    raw_html = simple_get('https://enzyme.expasy.org/EC/'+enzyme_class)
    html = BeautifulSoup(raw_html, 'html.parser')
    for i, tr in enumerate(html.select('tr')):
        if "Reaction catalysed" in tr.get_text():
            break;
    splitted = html.select('tr')[i+1].get_text().split("<=>")
    substrates = splitted[0]
    products = splitted[1]
    substrates = substrates.split(" + ")
    products = products.split(" + ")
    if len(substrates) > 2 or len(products) > 2:
        continue
    print("\"{}\": {}, {}".format(enzyme_class, substrates, products))
    

"2.1.1.185": ['S-adenosyl-L-methionine', 'guanosine(2251) in 23S rRNA '], [' S-adenosyl-L-homocysteine', "2'-O-methylguanosine(2251) in 23S rRNA"]
"2.1.1.206": ['S-adenosyl-L-methionine', 'cytidine(56) in tRNA '], [' S-adenosyl-L-homocysteine', "2'-O-methylcytidine(56) in tRNA"]
"2.1.1.220": ['S-adenosyl-L-methionine', 'adenine(58) in tRNA '], [' S-adenosyl-L-homocysteine', 'N(1)-methyladenine(58) in tRNA']
"2.1.1.228": ['S-adenosyl-L-methionine', 'guanine(37) in tRNA '], [' S-adenosyl-L-homocysteine', 'N(1)-methylguanine(37) in tRNA']
"2.1.1.257": ['S-adenosyl-L-methionine', 'pseudouridine(54) in tRNA '], [' S-adenosyl-L-homocysteine', 'N(1)-methylpseudouridine(54) in tRNA']
"2.1.1.33": ['S-adenosyl-L-methionine', 'guanine(46) in tRNA '], [' S-adenosyl-L-homocysteine', 'N(7)-methylguanine(46) in tRNA']
"2.1.1.45": ['5,10-methylenetetrahydrofolate', 'dUMP '], [' dihydrofolate', 'dTMP']
"2.1.1.98": ['3 S-adenosyl-L-methionine', '2-((3S)-3-carboxy-3-aminopropyl)-L-histidine-[translation 

## Padding test

In [71]:
s=tf.constant([[1.0,2.0,3.0],[4.0,5.0,6.0],[7.0,8.0,9.0]])

In [47]:
s1=tf.constant([[1.0,2.0,3.0],[4.0,5.0,6.0],[7.0,8.0,9.0],[10.0,11.0,12.0] ])

In [7]:
tf.pad(s, [[2,2],[2,2]], "SYMMETRIC")

<tf.Tensor: id=7, shape=(7, 7), dtype=float32, numpy=
array([[5., 4., 4., 5., 6., 6., 5.],
       [2., 1., 1., 2., 3., 3., 2.],
       [2., 1., 1., 2., 3., 3., 2.],
       [5., 4., 4., 5., 6., 6., 5.],
       [8., 7., 7., 8., 9., 9., 8.],
       [8., 7., 7., 8., 9., 9., 8.],
       [5., 4., 4., 5., 6., 6., 5.]], dtype=float32)>

In [9]:
tf.pad(s, [[2,2],[2,2]], "REFLECT")

<tf.Tensor: id=10, shape=(7, 7), dtype=float32, numpy=
array([[9., 8., 7., 8., 9., 8., 7.],
       [6., 5., 4., 5., 6., 5., 4.],
       [3., 2., 1., 2., 3., 2., 1.],
       [6., 5., 4., 5., 6., 5., 4.],
       [9., 8., 7., 8., 9., 8., 7.],
       [6., 5., 4., 5., 6., 5., 4.],
       [3., 2., 1., 2., 3., 2., 1.]], dtype=float32)>

In [72]:
s_copy = s

In [81]:
axis = 0


In [82]:
middle_point = s.get_shape().as_list()[axis]//2
middle_point

1

In [83]:
if axis == 0:
    b = s_copy[middle_point+1:, :]
    e = s_copy[:middle_point,:]
elif axis == 1:
    b = s_copy[:, middle_point+1:]
    e = s_copy[:, :middle_point]


In [84]:
tf.concat([b, s, e], axis=axis)

<tf.Tensor: id=299, shape=(5, 3), dtype=float32, numpy=
array([[7., 8., 9.],
       [1., 2., 3.],
       [4., 5., 6.],
       [7., 8., 9.],
       [1., 2., 3.]], dtype=float32)>

## Minibatch std discrimination

In [20]:
real=tf.ones([1,3,3], dtype=tf.float32)

In [22]:
real2 = real+tf.random_normal(real.shape, stddev=2.0)
real3 = real+tf.random_normal(real.shape, stddev=10.0)

In [23]:
real = tf.concat([real, real2, real3],axis=0)

In [24]:
real

<tf.Tensor: id=31, shape=(3, 3, 3), dtype=float32, numpy=
array([[[  1.       ,   1.       ,   1.       ],
        [  1.       ,   1.       ,   1.       ],
        [  1.       ,   1.       ,   1.       ]],

       [[ -1.320955 ,   1.7413759,  -0.4495827],
        [  0.3905062,   4.3406577,   0.6014135],
        [ -5.5202537,  -0.7708875,   1.141065 ]],

       [[  4.993128 , -12.313018 ,  11.608234 ],
        [ -3.090486 ,   5.05607  ,  -2.8435304],
        [ 23.344551 ,  -7.945652 ,  -2.3150027]]], dtype=float32)>

In [25]:
fake = tf.ones([3,3,3])
fake = fake + tf.random_normal(real.shape, stddev=0.001)

In [29]:
def minibatch_std(x):
    y = x
    y -= tf.reduce_mean(y, axis=0, keepdims=True)  # [GMCHW] Subtract mean over group.
    y = tf.reduce_mean(tf.square(y), axis=0)  # [MCHW]  Calc variance over group.
    y = tf.sqrt(y + 1e-8)  # [MCHW]  Calc stddev over group.
    y = tf.reduce_mean(y)  # [M111]  Take average of everything
    return y

In [41]:
real[:,0]

<tf.Tensor: id=100, shape=(3, 3), dtype=float32, numpy=
array([[  1.       ,   1.       ,   1.       ],
       [ -1.320955 ,   1.7413759,  -0.4495827],
       [  4.993128 , -12.313018 ,  11.608234 ]], dtype=float32)>

In [36]:
t = tf.reduce_mean(real, axis=0, keepdims=True)
t

<tf.Tensor: id=73, shape=(1, 3, 3), dtype=float32, numpy=
array([[[ 1.5573908 , -3.1905472 ,  4.0528836 ],
        [-0.5666599 ,  3.465576  , -0.414039  ],
        [ 6.2747655 , -2.5721798 , -0.05797919]]], dtype=float32)>

In [38]:
t2 = tf.reduce_mean(tf.square(t), axis=2)
t2

<tf.Tensor: id=81, shape=(1, 3), dtype=float32, numpy=array([[ 9.676974,  4.167583, 15.330716]], dtype=float32)>

In [31]:
print(minibatch_std(real))

tf.Tensor(4.1734023, shape=(), dtype=float32)


In [33]:
print(minibatch_std(fake))

tf.Tensor(0.00082576263, shape=(), dtype=float32)


In [54]:
to_add = tf.expand_dims(tf.concat([tf.zeros([3,1]), tf.ones([3,1]), tf.ones([3,1])*2], axis=1), axis=0)
to_add_full = tf.tile(to_add, multiples=[3,1,1]) 

In [55]:
(fake+to_add_full)[0]

<tf.Tensor: id=208, shape=(3, 3), dtype=float32, numpy=
array([[0.99790543, 1.9987406 , 2.9997768 ],
       [1.002098  , 1.9993657 , 3.0004382 ],
       [1.0022795 , 1.9996374 , 3.0006309 ]], dtype=float32)>

In [57]:
print(minibatch_std(fake+tf.transpose(to_add_full)))

tf.Tensor(0.81651014, shape=(), dtype=float32)


In [58]:
print(minibatch_std(fake+to_add_full))

tf.Tensor(0.0008257732, shape=(), dtype=float32)


In [156]:
def minibatch_stddev_layer_v2(x, group_size=1):
    with tf.variable_scope('MinibatchStddev'):
        group_size = tf.minimum(group_size,
                                tf.shape(x)[0])  # Minibatch must be divisible by (or smaller than) group_size.
        s = x.shape  # [NCHW]  Input shape.
        y = tf.reshape(x, [group_size, -1, s[1], s[2], s[3]])  # [GMCHW] Split minibatch into M groups of size G.
        y = tf.cast(y, tf.float32)  # [GMCHW] Cast to FP32.
        y -= tf.reduce_mean(y, axis=0, keepdims=True)  # [GMCHW] Subtract mean over group.
        y = tf.reduce_mean(tf.square(y), axis=0)  # [MCHW]  Calc variance over group.
        y = tf.sqrt(y + 1e-8)  # [MCHW]  Calc stddev over group.
        y = tf.reduce_mean(y, axis=[1, 2, 3], keepdims=True)  # [M111]  Take average over fmaps and pixels.
        #y = tf.cast(y, x.dtype)  # [M111]  Cast back to original data type.
        #y = tf.tile(y, [group_size, s[1], s[2], 1])  # [N1HW]  Replicate over group and pixels.
        return y#tf.concat([x, y], axis=3)  # [NHWC]  Append as new fmap.

In [157]:
#b = tf.random_uniform([4,1,2,3])
minibatch_stddev_layer(b)

<tf.Tensor: id=1666, shape=(4, 1, 2, 4), dtype=float32, numpy=
array([[[[0.65420175, 0.9305099 , 0.14062226, 0.2927614 ],
         [0.7431723 , 0.83801126, 0.71127343, 0.28765965]]],


       [[[0.7177837 , 0.2417531 , 0.8814992 , 0.2927614 ],
         [0.83159447, 0.56246233, 0.16293252, 0.28765965]]],


       [[[0.71108437, 0.6360712 , 0.2101903 , 0.2927614 ],
         [0.6388582 , 0.9094868 , 0.7958224 , 0.28765965]]],


       [[[0.4771614 , 0.06184614, 0.19554937, 0.2927614 ],
         [0.00622571, 0.8419572 , 0.26594615, 0.28765965]]]],
      dtype=float32)>

In [77]:
(0.65420175 + 0.9305099 +  0.14062226)/3 +(0.7177837 + 0.2417531 + 0.8814992)/3 + (0.71108437 + 0.6360712 + 0.2101903)/3 + (0.4771614 + 0.06184614 + 0.19554937)/3

1.9527575633333334

In [78]:
1.9527575633333334/4

0.48818939083333335

In [79]:
print(b)

tf.Tensor(
[[[[0.65420175 0.9305099  0.14062226]
   [0.7431723  0.83801126 0.71127343]]]


 [[[0.7177837  0.2417531  0.8814992 ]
   [0.83159447 0.56246233 0.16293252]]]


 [[[0.71108437 0.6360712  0.2101903 ]
   [0.6388582  0.9094868  0.7958224 ]]]


 [[[0.4771614  0.06184614 0.19554937]
   [0.00622571 0.8419572  0.26594615]]]], shape=(4, 1, 2, 3), dtype=float32)


In [158]:
a = tf.ones([4,1,2,3])


In [169]:
m = tf.expand_dims(tf.constant([1,2,3,4,5,6,7,8,1,2,3,4,5,6,7,8], dtype=tf.float32), axis=1)
m = tf.expand_dims(m, axis=2)
m = tf.expand_dims(m, axis=3)

In [170]:
c = tf.multiply(tf.ones([16,1,1,1]),m)
tf.squeeze(c)

<tf.Tensor: id=1788, shape=(16,), dtype=float32, numpy=
array([1., 2., 3., 4., 5., 6., 7., 8., 1., 2., 3., 4., 5., 6., 7., 8.],
      dtype=float32)>

In [171]:
minibatch_stddev_layer(c, group_size=2)

<tf.Tensor: id=1806, shape=(16, 1, 1, 2), dtype=float32, numpy=
array([[[[1.      , 2.291288]]],


       [[[2.      , 2.291288]]],


       [[[3.      , 2.291288]]],


       [[[4.      , 2.291288]]],


       [[[5.      , 2.291288]]],


       [[[6.      , 2.291288]]],


       [[[7.      , 2.291288]]],


       [[[8.      , 2.291288]]],


       [[[1.      , 2.291288]]],


       [[[2.      , 2.291288]]],


       [[[3.      , 2.291288]]],


       [[[4.      , 2.291288]]],


       [[[5.      , 2.291288]]],


       [[[6.      , 2.291288]]],


       [[[7.      , 2.291288]]],


       [[[8.      , 2.291288]]]], dtype=float32)>

<tf.Tensor: id=1588, shape=(16, 1, 2, 1), dtype=float32, numpy=
array([[[[1.],
         [1.]]],


       [[[1.],
         [1.]]],


       [[[1.],
         [1.]]],


       [[[1.],
         [1.]]],


       [[[2.],
         [1.]]],


       [[[2.],
         [1.]]],


       [[[2.],
         [1.]]],


       [[[2.],
         [1.]]],


       [[[3.],
         [1.]]],


       [[[3.],
         [1.]]],


       [[[3.],
         [1.]]],


       [[[3.],
         [1.]]],


       [[[3.],
         [1.]]],


       [[[3.],
         [1.]]],


       [[[3.],
         [1.]]],


       [[[3.],
         [1.]]]], dtype=float32)>

In [146]:
minibatch_stddev_layer(tf.concat([c, tf.ones([16,1,1,1])], axis=2))

<tf.Tensor: id=1610, shape=(16, 1, 2, 2), dtype=float32, numpy=
array([[[[1.000000e+00, 8.291562e-01],
         [1.000000e+00, 1.000000e-04]]],


       [[[1.000000e+00, 8.291562e-01],
         [1.000000e+00, 1.000000e-04]]],


       [[[1.000000e+00, 8.291562e-01],
         [1.000000e+00, 1.000000e-04]]],


       [[[1.000000e+00, 8.291562e-01],
         [1.000000e+00, 1.000000e-04]]],


       [[[2.000000e+00, 8.291562e-01],
         [1.000000e+00, 1.000000e-04]]],


       [[[2.000000e+00, 8.291562e-01],
         [1.000000e+00, 1.000000e-04]]],


       [[[2.000000e+00, 8.291562e-01],
         [1.000000e+00, 1.000000e-04]]],


       [[[2.000000e+00, 8.291562e-01],
         [1.000000e+00, 1.000000e-04]]],


       [[[3.000000e+00, 8.291562e-01],
         [1.000000e+00, 1.000000e-04]]],


       [[[3.000000e+00, 8.291562e-01],
         [1.000000e+00, 1.000000e-04]]],


       [[[3.000000e+00, 8.291562e-01],
         [1.000000e+00, 1.000000e-04]]],


       [[[3.000000e+00, 8.291562e-

In [101]:
m1 = tf.expand_dims(tf.constant([1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4], dtype=tf.float32), axis=1)
m1 = tf.expand_dims(m1, axis=2)
m1 = tf.expand_dims(m1, axis=3)

In [102]:
d = tf.multiply(tf.ones([16,1,2,3]),m1)
tf.squeeze(d)

<tf.Tensor: id=1115, shape=(16, 2, 3), dtype=float32, numpy=
array([[[1., 1., 1.],
        [1., 1., 1.]],

       [[2., 2., 2.],
        [2., 2., 2.]],

       [[3., 3., 3.],
        [3., 3., 3.]],

       [[4., 4., 4.],
        [4., 4., 4.]],

       [[1., 1., 1.],
        [1., 1., 1.]],

       [[2., 2., 2.],
        [2., 2., 2.]],

       [[3., 3., 3.],
        [3., 3., 3.]],

       [[4., 4., 4.],
        [4., 4., 4.]],

       [[1., 1., 1.],
        [1., 1., 1.]],

       [[2., 2., 2.],
        [2., 2., 2.]],

       [[3., 3., 3.],
        [3., 3., 3.]],

       [[4., 4., 4.],
        [4., 4., 4.]],

       [[1., 1., 1.],
        [1., 1., 1.]],

       [[2., 2., 2.],
        [2., 2., 2.]],

       [[3., 3., 3.],
        [3., 3., 3.]],

       [[4., 4., 4.],
        [4., 4., 4.]]], dtype=float32)>

In [103]:
minibatch_stddev_layer(a)

tf.Tensor(
[[[[1.]
   [1.]]]], shape=(1, 1, 2, 1), dtype=float32)
(4, 1, 2, 3)
(1, 1, 2, 1)
(1, 1, 2, 1)
(1, 1, 2, 1)


<tf.Tensor: id=1140, shape=(4, 1, 2, 4), dtype=float32, numpy=
array([[[[1.e+00, 1.e+00, 1.e+00, 1.e-04],
         [1.e+00, 1.e+00, 1.e+00, 1.e-04]]],


       [[[1.e+00, 1.e+00, 1.e+00, 1.e-04],
         [1.e+00, 1.e+00, 1.e+00, 1.e-04]]],


       [[[1.e+00, 1.e+00, 1.e+00, 1.e-04],
         [1.e+00, 1.e+00, 1.e+00, 1.e-04]]],


       [[[1.e+00, 1.e+00, 1.e+00, 1.e-04],
         [1.e+00, 1.e+00, 1.e+00, 1.e-04]]]], dtype=float32)>

In [106]:
c

<tf.Tensor: id=1102, shape=(16, 1, 1, 1), dtype=float32, numpy=
array([[[[1.]]],


       [[[1.]]],


       [[[1.]]],


       [[[1.]]],


       [[[2.]]],


       [[[2.]]],


       [[[2.]]],


       [[[2.]]],


       [[[3.]]],


       [[[3.]]],


       [[[3.]]],


       [[[3.]]],


       [[[4.]]],


       [[[4.]]],


       [[[4.]]],


       [[[4.]]]], dtype=float32)>

In [105]:
minibatch_stddev_layer(d)

tf.Tensor(
[[[[2.5]
   [2.5]]]], shape=(1, 1, 2, 1), dtype=float32)
(16, 1, 2, 3)
(1, 1, 2, 1)
(1, 1, 2, 1)
(1, 1, 2, 1)


<tf.Tensor: id=1190, shape=(16, 1, 2, 4), dtype=float32, numpy=
array([[[[1.      , 1.      , 1.      , 1.118034],
         [1.      , 1.      , 1.      , 1.118034]]],


       [[[2.      , 2.      , 2.      , 1.118034],
         [2.      , 2.      , 2.      , 1.118034]]],


       [[[3.      , 3.      , 3.      , 1.118034],
         [3.      , 3.      , 3.      , 1.118034]]],


       [[[4.      , 4.      , 4.      , 1.118034],
         [4.      , 4.      , 4.      , 1.118034]]],


       [[[1.      , 1.      , 1.      , 1.118034],
         [1.      , 1.      , 1.      , 1.118034]]],


       [[[2.      , 2.      , 2.      , 1.118034],
         [2.      , 2.      , 2.      , 1.118034]]],


       [[[3.      , 3.      , 3.      , 1.118034],
         [3.      , 3.      , 3.      , 1.118034]]],


       [[[4.      , 4.      , 4.      , 1.118034],
         [4.      , 4.      , 4.      , 1.118034]]],


       [[[1.      , 1.      , 1.      , 1.118034],
         [1.      , 1.      , 1.     

In [153]:
c = tf.expand_dims(tf.constant([1,2,3,4]), axis=1)

In [154]:
tf.reshape(c, [2,2,1])[0]

<tf.Tensor: id=1648, shape=(2, 1), dtype=int32, numpy=
array([[1],
       [2]])>

## Leaky relu

In [66]:
s=tf.constant([[1.0,-2.0,3.0,-4.0,5.0,-6.0],[-10.0,20.0,-30.0,40.0,-50.0,60.0]])

In [12]:
s

<tf.Tensor: id=16, shape=(2, 6), dtype=float32, numpy=
array([[  1.,  -2.,   3.,  -4.,   5.,  -6.],
       [-10.,  20., -30.,  40., -50.,  60.]], dtype=float32)>

In [13]:
tf.nn.leaky_relu(s)

<tf.Tensor: id=19, shape=(2, 6), dtype=float32, numpy=
array([[  1. ,  -0.4,   3. ,  -0.8,   5. ,  -1.2],
       [ -2. ,  20. ,  -6. ,  40. , -10. ,  60. ]], dtype=float32)>

In [14]:
tf.nn.relu(s)

<tf.Tensor: id=21, shape=(2, 6), dtype=float32, numpy=
array([[ 1.,  0.,  3.,  0.,  5.,  0.],
       [ 0., 20.,  0., 40.,  0., 60.]], dtype=float32)>

In [15]:
tf.layers.batch_normalization(s)

<tf.Tensor: id=72, shape=(2, 6), dtype=float32, numpy=
array([[  0.99950033,  -1.9990007 ,   2.998501  ,  -3.9980013 ,
          4.997502  ,  -5.9970026 ],
       [ -9.995004  ,  19.990007  , -29.98501   ,  39.980015  ,
        -49.97502   ,  59.970024  ]], dtype=float32)>

In [26]:
def lrelu(x, leak=0.2, name='lrelu'):
    """The leaky RELU operation."""
    with tf.variable_scope(name):
        f1 = 0.5 * (1 + leak)
        f2 = 0.5 * (1 - leak)
    return f1 * x + f2 * abs(x)

In [27]:
lrelu(s)

<tf.Tensor: id=138, shape=(2, 6), dtype=float32, numpy=
array([[  1.        ,  -0.40000004,   3.        ,  -0.8000001 ,
          5.        ,  -1.2       ],
       [ -2.        ,  20.        ,  -6.        ,  40.        ,
        -10.000002  ,  60.        ]], dtype=float32)>

In [29]:
tf.reduce_max(s)

<tf.Tensor: id=141, shape=(), dtype=float32, numpy=60.0>

In [48]:
tf.random_normal([3,3],
                mean=0.0,
                stddev=tf.reduce_max(s)/12,
                dtype=tf.float32)

<tf.Tensor: id=294, shape=(3, 3), dtype=float32, numpy=
array([[  5.1962004,  -2.9235325,  -3.9870965],
       [ -2.1220994, -11.072636 ,   5.929529 ],
       [  3.4704418,   2.6292017,  -5.3618007]], dtype=float32)>

## Pixel Normalization

In [49]:
x = tf.random_normal([3,3], mean=0.0,stddev=10,dtype=tf.float32)

In [50]:
x

<tf.Tensor: id=299, shape=(3, 3), dtype=float32, numpy=
array([[ -3.9545503,   4.832311 ,  -4.288933 ],
       [ -4.7606173,  -8.868431 ,  22.546158 ],
       [ 11.904067 , -12.1788225,  -3.4183855]], dtype=float32)>

In [52]:
epsilon=1e-8

In [53]:
normalized = x * tf.rsqrt(tf.reduce_mean(tf.square(x), axis=1, keepdims=True) + epsilon)

In [54]:
normalized

<tf.Tensor: id=310, shape=(3, 3), dtype=float32, numpy=
array([[-0.90419036,  1.1048865 , -0.9806455 ],
       [-0.33395392, -0.6221141 ,  1.5815969 ],
       [ 1.187017  , -1.2144144 , -0.34086516]], dtype=float32)>

In [55]:
tf.square(x)

<tf.Tensor: id=312, shape=(3, 3), dtype=float32, numpy=
array([[ 15.638468,  23.35123 ,  18.394945],
       [ 22.663477,  78.64907 , 508.32922 ],
       [141.70682 , 148.32372 ,  11.685359]], dtype=float32)>

In [57]:
tf.reduce_mean(tf.square(x), axis=1, keepdims=True)

<tf.Tensor: id=316, shape=(3, 1), dtype=float32, numpy=
array([[ 19.128214],
       [203.21393 ],
       [100.57196 ]], dtype=float32)>

In [59]:
(141.70682 + 148.32372 + 11.685359)/3

100.57196633333332

In [60]:
tf.rsqrt(tf.reduce_mean(tf.square(x), axis=1, keepdims=True) + epsilon)

<tf.Tensor: id=322, shape=(3, 1), dtype=float32, numpy=
array([[0.22864556],
       [0.07014929],
       [0.09971525]], dtype=float32)>

In [61]:
tf.sqrt(tf.reduce_mean(tf.square(x), axis=1, keepdims=True) + epsilon)

<tf.Tensor: id=328, shape=(3, 1), dtype=float32, numpy=
array([[ 4.3735814],
       [14.255312 ],
       [10.028557 ]], dtype=float32)>

## Max and min

In [29]:
z = tf.random_normal((3,3), stddev=1)

In [30]:
z

<tf.Tensor: id=66, shape=(3, 3), dtype=float32, numpy=
array([[-0.10148989,  1.039026  , -0.7246698 ],
       [-1.963244  , -0.97793996,  1.1597226 ],
       [-0.6023854 ,  0.9201034 , -1.3990489 ]], dtype=float32)>

In [67]:
greater_mask = z > 1
lower_mask = z < -1

In [52]:
tf.reduce_max(z)

<tf.Tensor: id=154, shape=(), dtype=float32, numpy=1.1597226>

In [62]:
tf.where(z > 1,  z +(1-tf.reduce_max(z)), z)

<tf.Tensor: id=214, shape=(3, 3), dtype=float32, numpy=
array([[-0.10148989,  0.87930346, -0.7246698 ],
       [-1.963244  , -0.97793996,  1.        ],
       [-0.6023854 ,  0.9201034 , -1.3990489 ]], dtype=float32)>

In [72]:
tf.where(z < -1,  z -(1+tf.reduce_min(z)), z)

<tf.Tensor: id=266, shape=(3, 3), dtype=float32, numpy=
array([[-0.10148989,  1.039026  , -0.7246698 ],
       [-1.        , -0.97793996,  1.1597226 ],
       [-0.6023854 ,  0.9201034 , -0.43580496]], dtype=float32)>

## Interpolation

In [63]:
import numpy as np
def slerp(val, low, high):
    """Code from https://github.com/soumith/dcgan.torch/issues/14"""
    omega = np.arccos(np.clip(np.dot(low/np.linalg.norm(low), high/np.linalg.norm(high)), -1, 1))
    so = np.sin(omega)
    if so == 0:
        return (1.0-val) * low + val * high # L'Hopital's rule/LERP
    return np.sin((1.0-val)*omega) / so * low + np.sin(val*omega) / so * high

In [68]:
for v in np.linspace(0, 1, 10):
    print(slerp(v, 1, 10))

1.0
2.0
3.0
4.0
5.0
6.0
6.999999999999999
7.999999999999999
9.0
10.0


In [66]:
np.linspace(0, 1, 10)

array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
       0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ])

## Random variation cost

In [4]:
fake=tf.ones([4,3,3], dtype=tf.float32)
real=tf.random_uniform([4,3,3], maxval=1.0, minval=-1.0)

In [5]:
real

<tf.Tensor: id=10, shape=(4, 3, 3), dtype=float32, numpy=
array([[[-0.68053555,  0.94120216,  0.5072081 ],
        [ 0.04000926,  0.7411902 ,  0.42095637],
        [-0.95797634, -0.3833294 ,  0.5742109 ]],

       [[ 0.2702098 , -0.23940134,  0.13490176],
        [-0.8861463 , -0.7361932 ,  0.7774217 ],
        [ 0.70588684,  0.03667521,  0.07178187]],

       [[ 0.09869027,  0.48066115,  0.46714878],
        [ 0.97773266,  0.8321655 , -0.42627954],
        [-0.9099207 , -0.38545132, -0.00822997]],

       [[ 0.49386597, -0.5003557 , -0.8435242 ],
        [-0.52237487,  0.24940586, -0.04144812],
        [-0.15424728, -0.35600853,  0.01232243]]], dtype=float32)>

In [8]:
def calculate_difference(batch):
    splitted = tf.split(tf.random_shuffle(batch),2)
    diff = tf.abs(splitted[0]-splitted[1])
    return tf.reduce_mean(diff)
    
    

In [15]:
calculate_difference(real)

<tf.Tensor: id=81, shape=(), dtype=float32, numpy=0.8252176>

In [11]:
calculate_difference(fake)

<tf.Tensor: id=45, shape=(), dtype=float32, numpy=0.0>

In [25]:
a = tf.ones((3, 3))
b = tf.ones((3, 1))

In [26]:
a,b

(<tf.Tensor: id=103, shape=(3, 3), dtype=float32, numpy=
 array([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=float32)>,
 <tf.Tensor: id=105, shape=(3, 1), dtype=float32, numpy=
 array([[1.],
        [1.],
        [1.]], dtype=float32)>)

In [27]:
a+b

<tf.Tensor: id=108, shape=(3, 3), dtype=float32, numpy=
array([[2., 2., 2.],
       [2., 2., 2.],
       [2., 2., 2.]], dtype=float32)>

In [29]:
tf.random_uniform((3,3), minval=-1, maxval=1
                )

<tf.Tensor: id=115, shape=(3, 3), dtype=float32, numpy=
array([[ 0.7145796 , -0.6244724 , -0.37418914],
       [ 0.92025685, -0.74070954,  0.08209896],
       [ 0.7871132 ,  0.7381878 ,  0.17941451]], dtype=float32)>

## Drop out

In [4]:
a = tf.ones((3, 3))

In [8]:
tf.nn.dropout(a, keep_prob=0.5)

<tf.Tensor: id=28, shape=(3, 3), dtype=float32, numpy=
array([[2., 2., 2.],
       [0., 2., 2.],
       [0., 2., 2.]], dtype=float32)>

In [12]:
a =tf.train.exponential_decay(1, 1, 100000, 0.05, staircase=True)


AttributeError: 'function' object has no attribute 'run'

## Swap

In [32]:
a = tf.identity([1,2,3,4,5])
b = tf.identity([-1,-2,-3,-4,-5])
l = tf.identity([10,20,30,40,50])

In [12]:
z = tf.random_uniform([5])

In [13]:
z

<tf.Tensor: id=18, shape=(5,), dtype=float32, numpy=
array([0.6608896 , 0.7125331 , 0.9374945 , 0.3813951 , 0.93090487],
      dtype=float32)>

In [15]:
z < 0.5

<tf.Tensor: id=22, shape=(5,), dtype=bool, numpy=array([False, False, False,  True, False])>

In [36]:
random_shuffle(a,b,l)

(<tf.Tensor: id=144, shape=(5,), dtype=int32, numpy=array([ 4,  3, -1, -5, -2])>,
 <tf.Tensor: id=150, shape=(5,), dtype=int32, numpy=array([-4, -3,  1,  5,  2])>,
 <tf.Tensor: id=156, shape=(5,), dtype=int32, numpy=array([40, 30, 10, 50, 20])>)

In [37]:
def random_shuffle(real, fake, labels, batch_size, num_real):
    idx = tf.random_shuffle(tf.range(int(batch_size)))
    real_idx = tf.gather(idx, tf.range(num_real))
    fake_idx = tf.gather(idx, tf.range(num_real, int(batch_size)))

    real_ = tf.gather(real, real_idx)
    fake_ = tf.gather(fake, fake_idx)
    real_mix = tf.concat([real_, fake_], axis=0)
    
    fake_ = tf.gather(fake, real_idx)
    real_ = tf.gather(real, fake_idx)
    fake_mix = tf.concat([fake_, real_], axis=0)
    
    labels1 = tf.gather(labels, real_idx)
    labels2 = tf.gather(labels, fake_idx)
    labels_mix = tf.concat([labels1, labels2], axis=0)
    
    return real_mix, fake_mix, labels_mix

## Upsampling

In [135]:
path ="C:\\Users\\Donatas\\Workspace\\Machine Learning\\SynBioAi\\PREnzyme\\data\\protein\\cgan\\sample_multi_51"

In [630]:
import os
def get_protein_batches(data_dir,
                        batch_size,
                        label_offset=0,
                        cycle_length=1,
                        shuffle_buffer_size=100000,
                        running_mode='train'):
    """Fetches num_batches batches of size batch_size from the data_dir.

    Args:
      data_dir: The directory to read data from. Expected to be a single
          TFRecords file.
      image_shape: The shape of image to be read
      batch_size: The number of elements in a single minibatch.
      num_batches: The number of batches to fetch at a time.
      label_offset: The scalar to add to the labels in the dataset. The
          GAN code expects labels in [0, 999], and this scalar can be used to move
          other labels into this range. (Default: 0)
      cycle_length: The number of input elements to process concurrently in the
          Dataset loader. (Default: 1)
      shuffle_buffer_size: The number of records to load before shuffling. Larger
          means more likely randomization. (Default: 100000)
    Returns:
      A list of num_batches batches of size batch_size.
    """
    # filenames = gfile.Glob(os.path.join(data_dir, '*_train_*-*-of-*'))
    print("Loading files from {}".format(data_dir))
    filenames = tf.gfile.Glob(os.path.join(data_dir, "test*.tfrecords"))
    print("Found {} file(s)".format(len(filenames)))
    filename_dataset = tf.data.Dataset.from_tensor_slices(filenames)
    filename_dataset = filename_dataset.shuffle(len(filenames))
    prefetch = max(int(batch_size / cycle_length), 1)
    dataset = filename_dataset.interleave(
        lambda fn: tf.data.TFRecordDataset(fn).prefetch(prefetch).repeat(),
        cycle_length=cycle_length, block_length=1)

    dataset = dataset.shuffle(buffer_size=shuffle_buffer_size, reshuffle_each_iteration=True)
    
    def _extract_seq_and_label(record):
        """Extracts and preprocesses the image and label from the record."""
        context_features, seq_features = tf.parse_single_sequence_example(
            serialized=record,
            context_features={
                'label': tf.FixedLenFeature([], tf.int64),
                'length': tf.FixedLenFeature([], tf.int64),
                #'sequence': tf.FixedLenFeature([], tf.int64)
              },
            sequence_features={
                'sequence': tf.FixedLenSequenceFeature([], dtype=tf.int64, allow_missing=False)
            }
        )
        seq = tf.cast(seq_features['sequence'], tf.int32, name="seq")
        def pad_up_to(t, max_in_dims, constant_values=tf.constant(0)):
            s = tf.shape(t)
            paddings = [[0, m-s[i]] for (i,m) in enumerate(max_in_dims)]
            return tf.pad(t, paddings, 'CONSTANT', constant_values=constant_values)
        length = tf.cast(context_features['length'], tf.int32)
        seq = pad_up_to(seq, [512])
        labels = tf.cast(context_features['label'], tf.int32, name="labels")
        labels += label_offset
        return length, labels
     
    dataset = dataset.map(_extract_seq_and_label, num_parallel_calls=16).prefetch(batch_size)   
  # Repeat for unlimited epochs.
    dataset = dataset.batch(batch_size, drop_remainder=True)



    iterator = dataset.make_one_shot_iterator()
    return iterator #iterator.get_next()

In [655]:
batch = get_protein_batches(path, 4, cycle_length=4, shuffle_buffer_size=4)

Loading files from C:\Users\Donatas\Workspace\Machine Learning\SynBioAi\PREnzyme\data\protein\cgan\sample_multi_51
Found 4 file(s)


In [773]:
test = batch.get_next()

In [598]:
import pandas as pd
DATA_SOURCE="C:\\Users\\Donatas\\Workspace\\Machine Learning\\SynBioAi\\PREnzyme\\data\\protein\\cgan\\data_sources\\augmented_2018_10_16.csv"
original_data = pd.read_csv(DATA_SOURCE, sep='\t', skipinitialspace=True).rename(columns={"EC class": "EC number"})[:2]

In [241]:
data = original_data
from common.bio.amino_acid import *
from common.bio.smiles import *
from common.preprocessing.dataframe import *
data["Sequence"] = from_amino_acid_to_id(data, "Sequence")
data["Label"] = 0
data = data.drop("EC number", axis =1)

In [242]:
filename = os.path.join(path, "test3.tfrecords")

with tf.python_io.TFRecordWriter(filename) as writer:
    i = 0
    for index, row in data.iterrows():
        features, label = row[0], row[1]
        example = tf.train.SequenceExample()
        example.context.feature["label"].int64_list.value.append(3)
        example.context.feature["length"].int64_list.value.append(i)
        feature_input = example.feature_lists.feature_list['sequence']
        # Then we store amino acid ids in 'input' feature (sequential data)
        for amino_acid in features:
            feature_input.feature.add().int64_list.value.append(amino_acid)
        i = i + 1
        writer.write(example.SerializeToString())
print("Data was stored in {}".format(filename))

Data was stored in C:\Users\Donatas\Workspace\Machine Learning\SynBioAi\PREnzyme\data\protein\cgan\sample_multi_51\test3.tfrecords


In [683]:
reaction_path = "../data/protein/cgan/sample_55/reactions_test.npy"

In [714]:
import numpy as np
reactions = np.load(reaction_path)

In [715]:
reactions

array([[0,
        list([12, 14, 6, 12, 7, 12, 12, 12, 6, 12, 12, 1, 15, 2, 12, 7]),
        list([15, 11, 12, 1, 18, 15, 5, 19, 2, 12, 1, 12, 15, 2, 20, 6, 20, 20, 20, 20, 20, 6]),
        list([12, 14, 6, 12, 7, 12, 12, 12, 6, 12, 12, 1, 15, 12, 1, 11, 15, 2, 12, 1, 12, 15, 2, 20, 6, 20, 20, 20, 20, 20, 6, 2, 12, 7]),
        list([15])],
       [1, list([3, 15, 12, 12, 1, 12, 15, 3, 2, 15, 3]),
        list([3, 12, 1, 11, 15, 2, 18, 15, 5, 19]),
        list([3, 12, 1, 11, 15, 2, 15, 12, 12, 1, 12, 15, 12, 1, 3, 2, 11, 15, 2, 15, 12, 1, 3, 2, 11, 15]),
        list([15])],
       [2,
        list([12, 20, 6, 20, 20, 1, 11, 15, 2, 22, 20, 7, 20, 20, 1, 15, 2, 20, 20, 20, 6, 7]),
        list([12, 12, 1, 11, 15, 2, 18, 15, 5, 19]),
        list([12, 12, 1, 11, 15, 2, 15, 20, 6, 20, 20, 20, 7, 20, 1, 12, 2, 20, 20, 1, 11, 15, 2, 22, 20, 7, 20, 6]),
        list([15])],
       [3, list([12, 18, 14, 4, 19, 1, 12, 2, 1, 12, 2, 12, 12, 15]),
        list([3, 12, 1, 11, 15, 2, 18, 15, 5, 19

In [716]:
def pad_up_to(input, max_in_dims, constant_values=tf.constant(0)):
    s = tf.shape(input)
    paddings = [[0, m - s[i]] for (i, m) in enumerate(max_in_dims)]
    return tf.pad(input, paddings, 'CONSTANT', constant_values=constant_values)

In [733]:
r1 = []
r1.append(1)

In [740]:
r1 = []
r2 = []
r3 = []
r4 = []
for reaction in reactions:
    label = tf.convert_to_tensor(reaction[0])
    r1.append(pad_up_to(tf.convert_to_tensor(reaction[1]), [128]))
    r2 = pad_up_to(tf.convert_to_tensor(reaction[2]), [128])
    r3 = pad_up_to(tf.convert_to_tensor(reaction[3]), [128])
    r4 = pad_up_to(tf.convert_to_tensor(reaction[4]), [128])
r1 = tf.stack(r1)   
r2 = tf.stack(r2)  
r3 = tf.stack(r3)  
r4 = tf.stack(r4)  

In [769]:
r1

<tf.Tensor: id=19587, shape=(61, 128), dtype=int32, numpy=
array([[12, 14,  6, ...,  0,  0,  0],
       [ 3, 15, 12, ...,  0,  0,  0],
       [12, 20,  6, ...,  0,  0,  0],
       ...,
       [15, 11, 20, ...,  0,  0,  0],
       [15, 20,  6, ...,  0,  0,  0],
       [12,  6,  1, ...,  0,  0,  0]])>

In [779]:
tf.expand_dims(test[1], axis=1)

<tf.Tensor: id=19679, shape=(4, 1), dtype=int32, numpy=
array([[3],
       [2],
       [3],
       [3]])>

In [778]:
tf.constant([[0],[1],[0]])

<tf.Tensor: id=19676, shape=(3, 1), dtype=int32, numpy=
array([[0],
       [1],
       [0]])>

In [780]:
tf.gather_nd(
    r1,
    tf.expand_dims(test[1], axis=1)) 

<tf.Tensor: id=19683, shape=(4, 128), dtype=int32, numpy=
array([[12, 18, 14,  4, 19,  1, 12,  2,  1, 12,  2, 12, 12, 15,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [12, 20,  6, 20, 20,  1, 11, 15,  2, 22, 20,  7, 20, 20,  1, 15,
         2, 20, 20, 20,  6,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
     

In [763]:
tf.squeeze([0,1,2])

<tf.Tensor: id=19630, shape=(3,), dtype=int32, numpy=array([0, 1, 2])>

In [753]:
reactions[[0,1,0]]

array([[0,
        list([12, 14, 6, 12, 7, 12, 12, 12, 6, 12, 12, 1, 15, 2, 12, 7]),
        list([15, 11, 12, 1, 18, 15, 5, 19, 2, 12, 1, 12, 15, 2, 20, 6, 20, 20, 20, 20, 20, 6]),
        list([12, 14, 6, 12, 7, 12, 12, 12, 6, 12, 12, 1, 15, 12, 1, 11, 15, 2, 12, 1, 12, 15, 2, 20, 6, 20, 20, 20, 20, 20, 6, 2, 12, 7]),
        list([15])],
       [1, list([3, 15, 12, 12, 1, 12, 15, 3, 2, 15, 3]),
        list([3, 12, 1, 11, 15, 2, 18, 15, 5, 19]),
        list([3, 12, 1, 11, 15, 2, 15, 12, 12, 1, 12, 15, 12, 1, 3, 2, 11, 15, 2, 15, 12, 1, 3, 2, 11, 15]),
        list([15])],
       [0,
        list([12, 14, 6, 12, 7, 12, 12, 12, 6, 12, 12, 1, 15, 2, 12, 7]),
        list([15, 11, 12, 1, 18, 15, 5, 19, 2, 12, 1, 12, 15, 2, 20, 6, 20, 20, 20, 20, 20, 6]),
        list([12, 14, 6, 12, 7, 12, 12, 12, 6, 12, 12, 1, 15, 12, 1, 11, 15, 2, 12, 1, 12, 15, 2, 20, 6, 20, 20, 20, 20, 20, 6, 2, 12, 7]),
        list([15])]], dtype=object)